## Imports

In [1]:
import pandas as pd
from tqdm import tqdm
tqdm.pandas()
import seaborn as sns
from nltk.tokenize import word_tokenize
import nltk

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/semcovici/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Read Data

In [3]:
path_raw_data = '../data/raw/'

In [4]:
data = pd.read_csv(
    path_raw_data + 'train_r2_bo_top_mentioned_timelines.csv', 
    sep = ';', 
    encoding='utf-8-sig'
    )

In [5]:
data

,User_ID,Polarity,Texts
0,r2_bo_1,for,Saudadinhaa dl :/ # Gente que beija com mordid...
1,r2_bo_1,against,Saudadinhaa dl :/ # Gente que beija com mordid...
2,r2_bo_1,against,Saudadinhaa dl :/ # Gente que beija com mordid...
3,r2_bo_3,for,"@Gremio E que domínio, hein campeão? # @Analis..."
4,r2_bo_3,for,"@Gremio E que domínio, hein campeão? # @Analis..."
...,...,...,...
7055,r2_bo_756,against,"Villas Boas manobra para não fazer de Mourão ""..."
7056,r2_bo_756,against,"Villas Boas manobra para não fazer de Mourão ""..."
7057,r2_bo_756,against,"Villas Boas manobra para não fazer de Mourão ""..."
7058,r2_bo_756,against,"Villas Boas manobra para não fazer de Mourão ""..."


## analysis

### Basic describe

In [6]:
#number of observations per label
data.Polarity.value_counts()

Polarity
against    4173
for        2887
Name: count, dtype: int64

In [7]:
print('len data with nan: ', data.shape)
print('len data without nan: ', data.dropna().shape)

len data with nan:  (7060, 3)
len data without nan:  (7060, 3)


In [8]:
print('len data with duplicates: ', data.shape)
print('len data without duplicates: ', data.drop_duplicates().shape)

len data with duplicates:  (7060, 3)
len data without duplicates:  (782, 3)


In [9]:
print('len data with duplicates: ', data['Texts'].shape)
print('len data without duplicates: ', data['Texts'].drop_duplicates().shape)

len data with duplicates:  (7060,)
len data without duplicates:  (471,)


In [12]:
data[data.duplicated(subset=['Texts'])]

,User_ID,Polarity,Texts
1,r2_bo_1,against,Saudadinhaa dl :/ # Gente que beija com mordid...
2,r2_bo_1,against,Saudadinhaa dl :/ # Gente que beija com mordid...
4,r2_bo_3,for,"@Gremio E que domínio, hein campeão? # @Analis..."
5,r2_bo_3,for,"@Gremio E que domínio, hein campeão? # @Analis..."
6,r2_bo_3,for,"@Gremio E que domínio, hein campeão? # @Analis..."
...,...,...,...
7055,r2_bo_756,against,"Villas Boas manobra para não fazer de Mourão ""..."
7056,r2_bo_756,against,"Villas Boas manobra para não fazer de Mourão ""..."
7057,r2_bo_756,against,"Villas Boas manobra para não fazer de Mourão ""..."
7058,r2_bo_756,against,"Villas Boas manobra para não fazer de Mourão ""..."


In [ ]:
data_2_anl = data.drop_duplicates(subset=['Texts'])
data_2_anl

In [ ]:
#number of observations per label
data_2_anl.Polarity.value_counts()

### numerical analysis of length of texts

In [ ]:
data_2_anl['list_texts'] = data_2_anl.Texts.progress_apply(lambda x: x.split(' # '))
data_2_anl.head()

In [ ]:
data_2_anl['n_texts'] = data_2_anl.list_texts.progress_apply(lambda x: len(x))
data_2_anl.head()

In [ ]:
data_2_anl['list_texts_tokens'] = data_2_anl.list_texts.progress_apply(lambda x: [word_tokenize(text, language='portuguese') for text in x])
data_2_anl.head()

In [ ]:
data_2_anl['list_texts_len'] = data_2_anl.list_texts_tokens.progress_apply(lambda x: [len(text) for text in x])
data_2_anl.head()

In [ ]:
sns.boxplot(
    data=data_2_anl,
    x="n_texts"
    ).set(
        title='number of comments of friends per user',
        xlabel='number of comments'
    )

In [ ]:
sns.boxplot(
    data=data_2_anl,
    x="n_texts",
    y='Polarity',
    hue='Polarity'
    ).set(
        title='number of comments of friends per user',
        xlabel='number of comments'
    )

In [ ]:
list_for_n_tks = []
list_against_n_tks = []
for idx, row in tqdm(data_2_anl.iterrows(), total = len(data_2_anl)):
    
    label = row['Polarity']
    list_texts_len = row['list_texts_len']
    
    if label == 'for':
        list_for_n_tks += list_texts_len
    elif label == 'against':
        list_against_n_tks += list_texts_len

In [ ]:
sns.boxplot(
    data={'against': list_against_n_tks, 'for': list_for_n_tks},
    orient = 'h',
    ).set(
        title='number of token in comments by polarity',
        xlabel='number of tokens'
    )